# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines = True)
df.head().T

,0
artist_id,AR8IEZO1187B99055E
artist_latitude,NaN
artist_location,
artist_longitude,NaN
artist_name,Marc Shaiman
duration,149.864
num_songs,1
song_id,SOINLJW12A8C13314C
title,City Slickers
year,2008


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = [x for x in df[['song_id','title','artist_id','year','duration']].values.squeeze()]
song_data

['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
import numpy as np

In [10]:
temp = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values.squeeze()
for i,x in enumerate(temp):
    try:
        if np.isnan(x):
            temp[i] = ''
    except TypeError:
        pass
artist_data = [x for x in temp]

In [11]:
artist_data = [x for x in temp]
artist_data

['AR8IEZO1187B99055E', 'Marc Shaiman', '', '', '']

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [13]:
log_files = get_files('data/log_data')

In [14]:
filepath = log_files[0]

In [26]:
df = pd.read_json(filepath, lines = True)
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [16]:
import datetime

In [17]:
df = df[df.page == 'NextSong']
df['ts'] = df['ts'].apply(lambda x : datetime.datetime.fromtimestamp(x//1000.0))
df.shape

(319, 18)

In [18]:
def extract(time):
    hour = time.hour
    day = time.day
    week  = time.isocalendar()[1]
    month= time.month
    year = time.year
    weekday = time.isocalendar()[2]
    return time, hour, day, week, month, year, weekday


def makedf(S, col):
    data = []
    for x in S.values:
        data.append(x)
    return pd.DataFrame(data, columns = col)
    

In [19]:
time_data = df['ts'].apply(extract)
column_labels = ('start_time','hour','day','week','month','year','weekday')

In [20]:
time_df = makedf(time_data,column_labels )
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-29 00:00:57,0,29,48,11,2018,4
1,2018-11-29 00:01:30,0,29,48,11,2018,4
2,2018-11-29 00:04:01,0,29,48,11,2018,4
3,2018-11-29 00:04:55,0,29,48,11,2018,4
4,2018-11-29 00:07:13,0,29,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [22]:
user_df = df[['userId','firstName','lastName', 'gender','level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid
2,24,Layla,Griffin,F,paid
3,73,Jacob,Klein,M,paid
4,24,Layla,Griffin,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [23]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [24]:
df[['ts','userId','level','sessionId','location','userAgent','song']].head(2)

,ts,userId,level,sessionId,location,userAgent,song
0,2018-11-29 00:00:57,73,paid,954,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Ain't No Sunshine
1,2018-11-29 00:01:30,24,paid,984,"Lake Havasu City-Kingman, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",My Advice 2 You (Explicit)


In [25]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplayid = index
    ts,userid,level, sessionid = row.ts, row.userId, row.level, row.sessionId
    location,user_agent = row.location, row.userAgent 
    
    songplay_data = (songplayid, ts, userid, level, songid, artistid, sessionid, location, user_agent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [27]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import numpy as np
import datetime



def process_song_file(cur, filepath):
    # open song file
    df = df = pd.read_json(filepath, lines = True)

    # insert song record
    song_data = [x for x in df[['song_id','title','artist_id','year','duration']].values.squeeze()]
    cur.execute(song_table_insert, song_data)
    
    # insert artist record
    temp = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values.squeeze()
    for i,x in enumerate(temp):
        try:
            if np.isnan(x):
                temp[i] = ''
        except TypeError:
            pass
    artist_data = [x for x in temp]
    cur.execute(artist_table_insert, artist_data)


def process_log_file(cur, filepath):
    def extract(time):
        hour = time.hour
        day = time.day
        week  = time.isocalendar()[1]
        month= time.month
        year = time.year
        weekday = time.isocalendar()[2]
        return time, hour, day, week, month, year, weekday


    def makedf(S, col):
        data = []
        for x in S.values:
            data.append(x)
        return pd.DataFrame(data, columns = col)

    # open log file
    df = pd.read_json(filepath, lines = True)

    # filter by NextSong action
    df = df[df.page == 'NextSong']
    # convert timestamp column to datetime
    df['ts'] = df['ts'].apply(lambda x : datetime.datetime.fromtimestamp(x//1000.0))
    t = df['ts']
    
    # insert time data records
    time_data = t.apply(extract)
    column_labels = ('start_time','hour','day','week','month','year','weekday')
    time_df = makedf(time_data,column_labels )


    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))

    # load user table
    user_df = df[['userId','firstName','lastName', 'gender','level']]

    # insert user records
    for i, row in user_df.iterrows():
        cur.execute(user_table_insert, row)

    # insert songplay records
    for index, row in df.iterrows():
        
        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()
        
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None

        # insert songplay record
        songplayid = index
        ts,userid,level, sessionid = row.ts, row.userId, row.level, row.sessionId
        location,user_agent = row.location, row.userAgent 
    
        songplay_data = (songplayid, ts, userid, level, songid, artistid, sessionid, location, user_agent)
        
        cur.execute(songplay_table_insert, songplay_data)
        
        
def process_data(cur, conn, filepath, func):
    # get all files matching extension from directory
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))

    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))

    # iterate over files and process
    for i, datafile in enumerate(all_files, 1):
        func(cur, datafile)
        conn.commit()
        print('{}/{} files processed.'.format(i, num_files))
        
def main():
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    cur = conn.cursor()

    process_data(cur, conn, filepath='data/song_data', func=process_song_file)
    process_data(cur, conn, filepath='data/log_data', func=process_log_file)

    conn.close()


if __name__ == "__main__":
    main()


71 files found in data/song_data
1/71 files processed.
2/71 files processed.
3/71 files processed.
4/71 files processed.
5/71 files processed.
6/71 files processed.
7/71 files processed.
8/71 files processed.
9/71 files processed.
10/71 files processed.
11/71 files processed.
12/71 files processed.
13/71 files processed.
14/71 files processed.
15/71 files processed.
16/71 files processed.
17/71 files processed.
18/71 files processed.
19/71 files processed.
20/71 files processed.
21/71 files processed.
22/71 files processed.
23/71 files processed.
24/71 files processed.
25/71 files processed.
26/71 files processed.
27/71 files processed.
28/71 files processed.
29/71 files processed.
30/71 files processed.
31/71 files processed.
32/71 files processed.
33/71 files processed.
34/71 files processed.
35/71 files processed.
36/71 files processed.
37/71 files processed.
38/71 files processed.
39/71 files processed.
40/71 files processed.
41/71 files processed.
42/71 files processed.
43/71 file